In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly
import xarray as xr
import netCDF4
import h5netcdf
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [7]:
ds = xr.open_dataset("C:/Users/shrey/OneDrive/Documents/IIITB/SEM 5/Data Visualization/Assignment-2/dataset/dataset/bi_2018.nc")
july_sept_ds = ds['burning_index_g'].sel(day=slice('2018-07-01', '2018-09-30'))
bi_data = july_sept_ds.isel(day=slice(None, None, 1))

In [9]:
print(bi_data)

<xarray.DataArray 'burning_index_g' (day: 92, lat: 585, lon: 1386)> Size: 597MB
[74594520 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 11kB -124.8 -124.7 -124.7 ... -67.14 -67.1 -67.06
  * lat      (lat) float64 5kB 49.4 49.36 49.32 49.28 ... 25.15 25.11 25.07
  * day      (day) datetime64[ns] 736B 2018-07-01 2018-07-02 ... 2018-09-30
Attributes:
    units:              Unitless
    description:        BI-G
    long_name:          bi
    standard_name:      bi
    dimensions:         lon lat time
    grid_mapping:       crs
    coordinate_system:  WGS84,EPSG:4326


In [10]:
null_count = bi_data.isnull().sum()
print(null_count, bi_data.size)

<xarray.DataArray 'burning_index_g' ()> Size: 8B
array(30077652) 74594520


In [12]:
# Check if all values are null for each day
all_nulls_per_day = bi_data.isnull().all(dim=['lat', 'lon'])

# Select only the days that do not have all null values
filtered_bi_data = bi_data.sel(day=~all_nulls_per_day)

filtered_bi_data

<xarray.DataArray 'burning_index_g' (day: 92, lat: 585, lon: 1386)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       ...,

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -67.14 -67.1 -67.06
  * lat      (lat) float64 49.4 49.36 49.32 49.28 ... 25.19 25.15 25.11 25.07
  * day      (day) datetime64[ns] 2018-07-01 2018-07-02 ... 2018-09-30
Attributes:
    units:              Unitless
    description:        BI-G
    long_name:          bi
    standard_name:      bi
    dimensions:         lon lat time
    grid_mapping:       crs
    coordinate_system:  WGS84,EPSG:4326

In [40]:
non_null_bi = bi_data.where(~bi_data.isnull(), drop=True)
unique_days = non_null_bi['day'].values
sampled_days = np.random.choice(unique_days, size=30, replace=False)
sampled_data = []
for day in sampled_days:
    # Select data for the sampled day
    day_data = bi_data.sel(day=day)
    sampled_data.append(day_data)

# Convert the list to an xarray DataArray or Dataset
sampled_data_array = xr.concat(sampled_data, dim='day')


In [41]:
sampled_data_array

<xarray.DataArray 'burning_index_g' (day: 30, lat: 585, lon: 1386)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -67.14 -67.1 -67.06
  * lat      (lat) float64 49.4 49.36 49.32 49.28 ... 25.19 25.15 25.11 25.07
  * day      (day) datetime64[ns] 2018-07-17 2018-08-14 ... 2018-09-29
Attributes:
    units:              Unitless
    description:        BI-G
    long_name:          bi
    standard_name:      bi
    dimensions:         lon lat time
    grid_mapping:       crs
    coordinate_system:  WGS84,EPSG:4326

In [8]:
# Get the data type of the 'burning_index_g' variable
burning_index_dtype = ds['burning_index_g'].dtype
print(f"Data type of 'burning_index_g': {burning_index_dtype}")

# Get the range of values for the 'burning_index_g' variable
burning_index_min = ds['burning_index_g'].min().item()
burning_index_max = ds['burning_index_g'].max().item()
print(f"Range of 'burning_index_g': {burning_index_min} to {burning_index_max}")

Data type of 'burning_index_g': float32
Range of 'burning_index_g': 0.0 to 180.0


In [20]:
bi_data

<xarray.DataArray 'burning_index_g' (day: 31, lat: 585, lon: 1386)>
[25135110 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 -124.8 -124.7 -124.7 -124.6 ... -67.14 -67.1 -67.06
  * lat      (lat) float64 49.4 49.36 49.32 49.28 ... 25.19 25.15 25.11 25.07
  * day      (day) datetime64[ns] 2018-07-01 2018-07-04 ... 2018-09-29
Attributes:
    units:              Unitless
    description:        BI-G
    long_name:          bi
    standard_name:      bi
    dimensions:         lon lat time
    grid_mapping:       crs
    coordinate_system:  WGS84,EPSG:4326

In [ ]:
def pre_process_data(ds, n_neighbors=5):
    
    df = ds['burning_index_g'].to_dataframe()

    imputer = KNNImputer(n_neighbors=n_neighbors)
    scaler=StandardScaler()
    df=scaler.fit_transform(df)

    imputed_values = imputer.fit_transform(df)

    df_imputed = pd.DataFrame(imputed_values, columns=df.columns, index=df.index)

    ds['burning_index_g'] = (('day', 'lat', 'lon'), df_imputed['burning_index_g'].values.reshape(ds['burning_index_g'].shape))

    return ds

code for imputing and sampling

In [21]:
reshaped_data = bi_data.stack(points=('day', 'lat', 'lon'))

In [22]:
reshaped_data_2d = reshaped_data.values
reshaped_data_2d = reshaped_data_2d.reshape(-1, 1)

In [23]:
imputer = KNNImputer(n_neighbors=5) 
imputed_data = imputer.fit_transform(reshaped_data_2d)
imputed_burning_index = xr.DataArray(
    imputed_data,  # Keep it as is (still in 2D)
    dims=reshaped_data.dims,
    coords=bi_data.coords
)



KeyboardInterrupt: 